In [1]:
import os 
import shutil
import whisper
from pydub import AudioSegment
import pandas as pd
import numpy as np

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
def split_audio_into_segments(audio_path, destination_dir):
    """
    Transcribes and splits audio into segments
    Args:
        audio_path (str): the path of the audio file
        destination_dir (str): parent directory in which splitted files are saved
    Returns:
        output_dir (str): folder directory in which splitted files are saved
        texts (list): a list of texts that splitted by segmentation
    """
    # Transcribing the audio and get word timestamps using Whisper
    result = model.transcribe(audio_path, word_timestamps=True)
    # Loading the original audio file
    audio = AudioSegment.from_file(audio_path)

    # Iterating through words and split at the specified word
    texts = []
    # segment_end_time_list = []
    start_time = np.float64(0.0)
    for i in range(len(result['segments'])):
        segment = result['segments'][i]
        texts.append(segment['text'])
        end_time = segment['end']
        new_audio = audio[start_time:end_time*1000]
        start_time = end_time*1000
        if i == 0:
            output_dir = os.path.join(destination_dir, audio_path.rsplit('/', 1)[1].rsplit('.', 1)[0])
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
        output_path = os.path.join(output_dir, f"segment_{i+1}.mp3")
        new_audio.export(output_path, format="mp3")
        if (i+1) == len(result['segments']):
            print(f"Exported ({len(result['segments'])}/{len(result['segments'])}): {output_dir}")
        # segment_end_time_list.append(end_time)
    return output_dir, texts

In [4]:
def split_audio_into_segments(audio_path, destination_dir, segment_max_time=30.00):
    """
    Transcribes and splits audio into segments depending on segment_max_time
    Args:
        audio_path (str): the path of the audio file
        destination_dir (str): parent directory in which splitted files are saved
        segment_max_time (float): the maximum time of a segment (sec)
    Returns:
        output_dir (str): folder directory in which splitted files are saved
        data_texts (list): a list of texts that splitted by segmentation
    """
    # Transcribing the audio and get word timestamps using Whisper
    result = model.transcribe(audio_path, word_timestamps=True)
    # Loading the original audio file
    audio = AudioSegment.from_file(audio_path)

    output_dir = os.path.join(destination_dir, audio_path.rsplit('/', 1)[1].rsplit('.', 1)[0])
    # Making the output directory if it is not existed
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)        
    
    os.makedirs(output_dir)

    # segment_end_time_list = []
    start_time = np.float64(0.0)
    segment_num = 1
    data_texts = []
    sub_texts = []
    for i in range(len(result['segments'])):
        segment = result['segments'][i]
        
        if (segment['end'] - start_time) < np.float64(segment_max_time):
            sub_texts.append(segment['text'])
            end_time = segment['end']
        else:
            if not sub_texts:
                data_texts.append(segment['text'])  
                new_audio = audio[start_time*1000:segment['end']*1000]   
                # print(f"segment_{segment_num}: {segment['end']}")
                output_path = os.path.join(output_dir, f"segment_{segment_num}.mp3")
                new_audio.export(output_path, format="mp3")
                
                start_time = segment['end']
                segment_num += 1
                continue
            else:                
                data_texts.append("".join(sub_texts))  
                new_audio = audio[start_time*1000:end_time*1000]                
                output_path = os.path.join(output_dir, f"segment_{segment_num}.mp3")
                new_audio.export(output_path, format="mp3")
                # print(f"segment_{segment_num}: {segment['end']}")
                
                sub_texts = [segment['text']]
                start_time = segment['start']
                segment_num += 1

        if (i+1) == len(result['segments']):
            data_texts.append("".join(sub_texts)) 
            new_audio = audio[start_time*1000:]                
            output_path = os.path.join(output_dir, f"segment_{segment_num}.mp3")
            new_audio.export(output_path, format="mp3")
            # print(f"segment_{segment_num}: {segment['end']}")
            print(f"Exported ({segment_num}/{segment['end']}): {output_dir}")
        # segment_end_time_list.append(end_time)
    return output_dir, data_texts

In [5]:
model = whisper.load_model("small")
segment_max_time = 60.00

In [6]:
audio_files_dir = "data/original/phishing"
audio_files = [os.path.join(audio_files_dir, f) for f in os.listdir(audio_files_dir) if f.endswith(".mp3")]

destination_dir = "data/splitted/phishing"

In [7]:
data_phishing_text = []
for i in range(len(audio_files)):
    audio_path = audio_files[i]
    print(f"*********** Splitting {i+1}/{len(audio_files)} ***********")
    output_dir, texts = split_audio_into_segments(audio_path, destination_dir, segment_max_time = segment_max_time)
    data_phishing_text.append([output_dir, texts])

*********** Splitting 1/506 ***********
Exported (1/53.64): data/splitted/phishing/data_agency_71
*********** Splitting 2/506 ***********
Exported (5/253.64): data/splitted/phishing/data_mp4_64
*********** Splitting 3/506 ***********
Exported (11/613.06): data/splitted/phishing/data_agency_1
*********** Splitting 4/506 ***********
Exported (3/150.64): data/splitted/phishing/data_agency_76
*********** Splitting 5/506 ***********
Exported (2/114.42): data/splitted/phishing/data_loan_142
*********** Splitting 6/506 ***********
Exported (7/373.98): data/splitted/phishing/data_mp4_73
*********** Splitting 7/506 ***********
Exported (4/207.68): data/splitted/phishing/data_loan_75
*********** Splitting 8/506 ***********
Exported (3/147.9): data/splitted/phishing/data_loan_164
*********** Splitting 9/506 ***********
Exported (22/1251.7): data/splitted/phishing/data_agency_29
*********** Splitting 10/506 ***********
Exported (3/123.78): data/splitted/phishing/data_mp4_48
*********** Splitting 1

In [8]:
audio_files_dir = "data/original/nonphishing"
audio_files = [os.path.join(audio_files_dir, f) for f in os.listdir(audio_files_dir) if f.endswith(".mp3")]

destination_dir = "data/splitted/nonphishing"

In [9]:
data_nonphishing_text = []
for i in range(len(audio_files)):
    audio_path = audio_files[i]
    print(f"*********** Splitting {i+1}/{len(audio_files)} ***********")
    output_dir, texts = split_audio_into_segments(audio_path, destination_dir, segment_max_time = segment_max_time)
    data_nonphishing_text.append([output_dir, texts])

*********** Splitting 1/500 ***********
Exported (3/139.52): data/splitted/nonphishing/data_subscription_cancellation_81
*********** Splitting 2/500 ***********
Exported (3/131.72): data/splitted/nonphishing/data_loan_service_25
*********** Splitting 3/500 ***********
Exported (2/100.72): data/splitted/nonphishing/data_subscription_cancellation_157
*********** Splitting 4/500 ***********
Exported (2/111.46): data/splitted/nonphishing/data_transaction_hist_34
*********** Splitting 5/500 ***********
Exported (3/121.66): data/splitted/nonphishing/data_loan_service_112
*********** Splitting 6/500 ***********
Exported (3/148.66): data/splitted/nonphishing/data_subscription_cancellation_83
*********** Splitting 7/500 ***********
Exported (3/122.32): data/splitted/nonphishing/data_subscription_cancellation_23
*********** Splitting 8/500 ***********
Exported (2/119.04): data/splitted/nonphishing/data_loan_service_102
*********** Splitting 9/500 ***********
Exported (2/110.68): data/splitted/no

In [10]:
df_phishing_text = pd.DataFrame(data_phishing_text, columns = ["Audio_file_path", "Text"])
df_phishing_text.head()

,Audio_file_path,Text
0,data/splitted/phishing/data_agency_71,[ 지금 전화 연락드렸어요. 지금 통화 가능하십니까? 네. 네. 지금 포닉명의가 지...
1,data/splitted/phishing/data_mp4_64,[ 올레오 quantity洗질의 종일 정오 이를 치열한 또 이렇게 안정 계좌에 리퀴...
2,data/splitted/phishing/data_agency_1,[ 그리고 현지 본인과 저희가 공부했던 박성수 일단 관련성이 밝혀지지 않다 보니까 ...
3,data/splitted/phishing/data_agency_76,[ 일단 이 의회에 8명을 건거했는데요. 그 건거 당시 현장에서 대량의 대포 통장과...
4,data/splitted/phishing/data_loan_142,[ 네 여보세요? 네 여보세요? 네네 아 예 선생님 일단 확인 됐구요 아 예 그 발...


In [11]:
path = "data/input_features/text/df_phishing_text.csv"
df_phishing_text.to_csv(path, index=False)

In [12]:
df_nonphishing_text = pd.DataFrame(data_nonphishing_text, columns = ["Audio_file_path", "Text"])
df_nonphishing_text.head()

,Audio_file_path,Text
0,data/splitted/nonphishing/data_subscription_ca...,[ 주택용 화재보험 가입하려고 하는데요. 생각해놓으신 특약이나 금액 한도 같은 게 ...
1,data/splitted/nonphishing/data_loan_service_25,[ 직장 없어도 대출 되나요? 직장이 없으면 아무래도 대출의 제약이 있습니다 그럼 ...
2,data/splitted/nonphishing/data_subscription_ca...,[ 통장 해지 않은 방법을 알 수 있을까요? 개인이신가요? 개인 통장입니다. 개인 ...
3,data/splitted/nonphishing/data_transaction_his...,"[ 여보세요, 뭐요 주시기 얼마인지 궁금합니다. 사랑합니다, 고객님. 0000 증권..."
4,data/splitted/nonphishing/data_loan_service_112,[ 신용대출을 받고 싶은데 어떻게 신청합니까? 신용대출은 개인 신형 평가 시스템에 ...


In [13]:
path = "data/input_features/text/df_nonphishing_text.csv"
df_nonphishing_text.to_csv(path, index=False)

In [21]:
df_phishing_text.shape

(506, 2)